## Get Albums and Artist Info from Spotify

In [16]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import re
import pandas as pd
import numpy as np
import sys

#These are my personal spotify credentials to be able to access the spotify API and ultimately access their data
clientId = '7cccb2739a114d25aa4e29b80f9af243'
clientSecret = 'd25f040f2f754174b979a492e7edb663'
clientCredentialsManager = SpotifyClientCredentials(client_id=clientId, client_secret=clientSecret)
sp = spotipy.Spotify(client_credentials_manager=clientCredentialsManager) #spotify object to access API

def spotifyArtistInfo(artist):
    result = sp.search(artist,type='artist') #search query
    artist = result['artists']['items'][0]
    followers = artist['followers']['total']
    genres = artist['genres']
    return (followers, genres)

def spotifyArtistAblums(artist):
    search = sp.search(q='artist:' + artist, type='artist')
    artist = search['artists']['items'][0]
    albums = []
    search = sp.artist_albums(artist['id'], album_type='album')
    albums.extend(search['items'])
    while search['next']:
        search = sp.next(search)
        albums.extend(search['items'])
    studioAlbums = []
    allAlbums = []
    albumNames = set()
    albums.sort(key=lambda album:album['name'].lower())
    firstAlbYear = 3000 #Arbitrarilly large
    for album in albums:
        name = album['name']
        albumId = album['id']
        year = int(album['release_date'][0:4])
        if year < firstAlbYear:
            firstAlbYear = year
        type_ = album['album_group']
        #If there is an album that has a deluxe version, the deluxe one will be selected
        exclude = ["spotify sessions","radio release", "karaoke","extended edition", "commentary", " tour edition", "remix album", "standard version", "international edition","international version", "international deluxe", "anniversary edition", "special edition", " case version", "bonus version", "asia deluxe", "japan deluxe", "china version", "japan version", "japanese version", "us version","uk version","canada version", "int'l version","(international", "international)", "soundtrack", "radio release", "deluxe package", "(clean)", "clean version"]
        if name not in albumNames and not any(x in name.lower() for x in exclude) and not ("edition" in name.lower() and "deluxe" not in name.lower()) and type_ == 'album':
            albumNames.add(name)
            allAlbums.append((name, album))
    notDeluxe = []
    deluxeVersion = []
    unDeluxed = []
    terms = ["(deluxe)", "(deluxe edition)", "(deluxe version)", " deluxe edition", " deluxe version", "deluxe"]
    for a in allAlbums:
        if "deluxe" in a[0].lower():
            deluxeVersion.append(a)
            studioAlbums.append(a[1])
            deluxeName = a[0].lower()
            for term in terms:
                deluxeName = deluxeName.replace(term, "")
            unDeluxed.append(deluxeName)
        else:
            notDeluxe.append(a)
    for item in notDeluxe:
        #print(unDeluxed)
        if not any(d.strip().lower() == item[0].strip().lower() for d in unDeluxed):
            studioAlbums.append(item[1])
    for i in studioAlbums:
        #Toggle this on to print
        #print(i["name"])
        pass
    return (studioAlbums, firstAlbYear)
        

In [2]:
albs, firstYear = spotifyArtistAblums("Ed Sheeran")

## Run Qualifier Check for All Billboard Artists

An artist qualifies for consideration in the study if:
- they have content which has appeared on the Billboard Hot 100 in the last 20 years
- they they have at least 3 albums released
- their first album was released at least 7 years ago

The billboard scraper goes back 1043 weeks (20 years) to collect data, and then this script will use spotify to ultimately see if the artist qualifies.

In [ ]:
#Currently doesn't catch destiny's child
dataPre = pd.DataFrame(pd.read_csv('Data/Weekly_data_clean.csv'))
artistData = pd.DataFrame(columns = ["Artist", 'Followers', "Genres", "NumAlbums", "YearFirstAlbum"])
drops = []
outArtists = []
for index, row in dataPre.iterrows():
    artists = row["Artists"].split(",")
    ##Some special cases
    if artists == ['Machine Gun Kelly', '', ' Ambassadors', ' Bebe Rexha']:
        artists = ['Machine Gun Kelly', 'X Ambassadors', ' Bebe Rexha']
    elif artists == ['Pusha T Kanye West']:
        artists = ["Pusha T", "Kanye West"]
    elif artists == ['Jay Z Kanye West']:
        artists = ['Jay Z', "Kanye West"]
    elif 'Tyler' in artists and ' The Creator' in artists:
        artists.remove("Tyler")
        artists.remove(" The Creator")
        artists.append("Tyler The Creator")
    elif " Swae Lee Or Rae Sremmurd" in artists:
        artists = ["Swae Lee", "Rae Sremmurd"]
    elif "Plies Or OJ Da Juiceman" in artists:
        artists = ['Plies', 'OJ Da Juiceman']
    if all(a.strip() in (artistData['Artist'].tolist() + outArtists) for a in artists):
        for a in artists:
            if a.strip() in outArtists:
                drops.append(index)
        continue
    else:
        for a in artists:
            artist = a.strip().replace(" &", "")
            if " + " in artist and artist !=  "Dan + Shay":
                new = artist.split("+")
                artists.extend(new)
                continue
            elif " with " in artist:
                new = artist.split(' with ')
                artists.extend(new)
                continue
            elif " With " in artist:
                new = artist.split(' With ')
                artists.extend(new)
                continue
            if artist in outArtists:
                drops.append(index)
                continue
            if artist not in artistData['Artist'].tolist():
                #print(str(index) + " ARTIST : " + artist)
                try:
                    albums, firstYear= spotifyArtistAblums(artist)
                except: 
                    #Add to this as we find
                    transferArtists = {"DU DDU":"BLACKPINK", "NBA YoungBoy":"YoungBoy Never Broke Again","Maejor Ali":"Maejor", "ROCKMAFIA":"rock mafia", "Soulja Boy Tell'em":"Soulja Boy", "Ludacris Co":"Ludacris", "Purple Ribbon All":"Purple Ribbon All-Stars", "Fat Man Scoop":"Fatman Scoop", "The Clark Family Experience":"Clark Family Experience", "SoulDecision":"Soul Decision", "Travi$ Scott": "Travis Scott" }
                    if artist in list(transferArtists.keys()):
                        artist = transferArtists[artist]
                    try:
                        albums, firstYear= spotifyArtistAblums(artist)
                    except:
                        #print("Not actually a spotify artist!")
                        continue
                if len(albums) >= 3 and firstYear < 2012: #then they qualify, and collect the artist data
                    followers, genres = spotifyArtistInfo(artist)
                    artistData = artistData.append({"Artist": artist, 'Followers':followers, "Genres": ",".join(genres), "NumAlbums":len(albums), "YearFirstAlbum": firstYear}, ignore_index = True) 
                else:
                    outArtists.append(artist)
                    drops.append(index)
        
dataPost = dataPre.drop(drops)
dataPost.to_csv("Data/billboardRankings.csv")
artistData.to_csv("Data/artistInfo.csv")
        
        

# Attaining Spotify's Audio Features

In [17]:
def trackInfo(albums, artist):
    albumData = pd.DataFrame(columns = ['Artist', 'Album','Name', 'Popularity', 'Duration', 'Explicit', 'Danceability', 'Energy', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'TimeSignature'])
    for album in albums:
        albumId = album['id']
        albumName = album['name']
        tracks = sp.album_tracks(albumId)['items']
        for track in tracks:
            trackName = track['name']
            features = sp.audio_features(track['uri'])
            #to get popularity and explicit
            trackInfo = sp.track(track['uri'])
            #Create DF row     ###Maybe try to change duration to be in seconds
            albumData = albumData.append({'Artist':artist, 'Album':albumName,'Name':trackName, 'Popularity':trackInfo['popularity'], 'Duration':features[0]['duration_ms'], 'Explicit':trackInfo['explicit'], 'Danceability':features[0]['danceability'], 'Energy':features[0]['energy'], 'Loudness':features[0]['loudness'], 'Mode':features[0]['mode'], 'Speechiness':features[0]['speechiness'], 'Acousticness':features[0]['acousticness'], 'Instrumentalness':features[0]['instrumentalness'], 'Liveness':features[0]['liveness'], 'Valence':features[0]['valence'], 'Tempo':features[0]['tempo'], 'TimeSignature':features[0]['time_signature']}, ignore_index = True)
    return albumData

In [4]:
trackInfo(albs, 'Ed Sheeran')

,Artist,Album,Name,Popularity,Duration,Explicit,Danceability,Energy,Loudness,Mode,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,TimeSignature
0,Ed Sheeran,x (Deluxe Edition),One,67,252760,False,0.464,0.321,-11.120,1,0.0418,0.877000,0,0.0789,0.306,93.528,4
1,Ed Sheeran,x (Deluxe Edition),I'm a Mess,70,244573,False,0.697,0.377,-7.755,1,0.0397,0.556000,0,0.0999,0.336,138.754,4
2,Ed Sheeran,x (Deluxe Edition),Sing,71,235382,False,0.818,0.670,-4.451,0,0.0472,0.304000,1.22e-06,0.0601,0.939,119.988,4
3,Ed Sheeran,x (Deluxe Edition),Don't,74,219840,False,0.806,0.608,-7.008,1,0.0659,0.011300,0,0.6350,0.849,95.049,4
4,Ed Sheeran,x (Deluxe Edition),Nina,61,225533,False,0.791,0.669,-7.189,0,0.1460,0.175000,0,0.3300,0.809,92.013,4
5,Ed Sheeran,x (Deluxe Edition),Photograph,85,258987,False,0.614,0.379,-10.480,1,0.0476,0.607000,0.000464,0.0986,0.201,107.989,4
6,Ed Sheeran,x (Deluxe Edition),Bloodstream,70,300253,False,0.660,0.316,-11.567,0,0.0364,0.529000,0.000294,0.1040,0.543,91.207,4
7,Ed Sheeran,x (Deluxe Edition),Tenerife Sea,72,241347,False,0.530,0.346,-10.497,1,0.0376,0.697000,1.07e-05,0.1050,0.359,121.876,4
8,Ed Sheeran,x (Deluxe Edition),Runaway,61,205133,False,0.767,0.449,-7.158,0,0.0364,0.024000,0.000299,0.1510,0.908,95.030,4
9,Ed Sheeran,x (Deluxe Edition),The Man,59,250427,False,0.759,0.725,-7.006,0,0.1580,0.576000,0,0.0771,0.619,87.467,4


## Write the Files For Archetype Artists

In [18]:
def archetypeContentInfo(artistList):
    df = pd.DataFrame(columns = ['Artist', 'Album', 'Popularity', 'Explicit', 'Danceability', 'Energy', 'Loudness', 'Mode', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration', 'TimeSignature'])
    for artist in artistList:
        #this is for testing
        if artistList.index(artist) > 5:
            df.to_csv("Data/songAttributes.csv")
            sys.exit()
        followers, genres = spotifyArtistInfo(artist)
        albums, firstYear = spotifyArtistAblums(artist)
        print("on artist " + str(artistList.index(artist)) + "out of 1168")
        try:
            data = trackInfo(albums, artist)
            df = pd.concat([data, df])
        except:
            pass
    df.to_csv("Data/songAttributes.csv")
    

In [19]:
artistList = pd.read_csv("Data/artistInfo.csv",names = ["Artist", "Followers", "Genres", "NumAlbums", "YearFirstAlbum"]).Artist.to_list()[1:]

archetypeContentInfo(artistList)

on artist 0out of 1168


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  del sys.path[0]


on artist 1out of 1168
on artist 2out of 1168
on artist 3out of 1168
on artist 4out of 1168
on artist 5out of 1168


SystemExit: 